In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from linearmodels.panel import PanelOLS
from linearmodels.panel import compare
import datetime


# 데이터 읽기 및 전처리
df_topic_day = pd.read_csv("../../R/data/topic_day_panel.csv")
sede_treat_0 = pd.read_csv("../../data/treat_0.csv")
sede_treat_1 = pd.read_csv("../../data/treat_1.csv")

sede_treat_0_1 = pd.read_csv("../../data/treat_0_1.csv")
sede_treat_0_2 = pd.read_csv("../../data/treat_0_2.csv")
sede_treat_1_1 = pd.read_csv("../../data/treat_1_1.csv")
sede_treat_1_2 = pd.read_csv("../../data/treat_1_2.csv")

In [2]:
df_topic_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16100 entries, 0 to 16099
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            16100 non-null  object 
 1   tag             16100 non-null  object 
 2   q_count         16100 non-null  int64  
 3   treat           16100 non-null  int64  
 4   post            16100 non-null  int64  
 5   rel_day         16100 non-null  int64  
 6   rel_week        16100 non-null  int64  
 7   subreddit_name  9982 non-null   object 
 8   subs            9387 non-null   float64
 9   posts           8316 non-null   float64
 10  comments        9044 non-null   float64
 11  rel_month       16100 non-null  int64  
 12  treated         16100 non-null  int64  
 13  fect_id         16100 non-null  int64  
 14  ln_q_count      16100 non-null  float64
dtypes: float64(4), int64(8), object(3)
memory usage: 1.8+ MB


In [3]:
df_topic_day['date'] = pd.to_datetime(df_topic_day['date'])

In [4]:
df_topic_day[['date', 'rel_week', 'rel_day']].drop_duplicates().to_csv('./chk.csv')

In [5]:
df_topic_day[(df_topic_day['date'] >='2021-10-06')&(df_topic_day['date'] <='2021-10-10')&(df_topic_day['tag'] =='.net')]

,date,tag,q_count,treat,post,rel_day,rel_week,subreddit_name,subs,posts,comments,rel_month,treated,fect_id,ln_q_count
0,2021-10-06,.net,48,0,0,-56,-8,dotnet,102578.0,26.0,150.0,-2,0,1,3.871201
1,2021-10-07,.net,45,0,0,-55,-8,dotnet,102621.0,11.0,95.0,-2,0,1,3.806662
2,2021-10-08,.net,41,0,0,-54,-8,dotnet,102654.0,14.0,97.0,-2,0,1,3.713572
3,2021-10-09,.net,17,0,0,-53,-8,dotnet,102688.0,8.0,119.0,-2,0,1,2.833213
4,2021-10-10,.net,13,0,0,-52,-8,dotnet,102729.0,18.0,90.0,-2,0,1,2.564949


In [6]:
sede_treat_0_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 52 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cdate                174 non-null    object
 1   del_yn               174 non-null    object
 2   net                  174 non-null    int64 
 3   amazon_web_services  174 non-null    int64 
 4   android              174 non-null    int64 
 5   angular              174 non-null    int64 
 6   api                  174 non-null    int64 
 7   arrays               174 non-null    int64 
 8   azure                174 non-null    int64 
 9   c                    174 non-null    int64 
 10  c#                   174 non-null    int64 
 11  cplusplus            174 non-null    int64 
 12  css                  174 non-null    int64 
 13  dart                 174 non-null    int64 
 14  dataframe            174 non-null    int64 
 15  django               174 non-null    int64 
 16  docker  

In [7]:
sede_treat_0_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 52 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cdate                148 non-null    object
 1   del_yn               148 non-null    object
 2   net                  148 non-null    int64 
 3   amazon_web_services  148 non-null    int64 
 4   android              148 non-null    int64 
 5   angular              148 non-null    int64 
 6   api                  148 non-null    int64 
 7   arrays               148 non-null    int64 
 8   azure                148 non-null    int64 
 9   c                    148 non-null    int64 
 10  c#                   148 non-null    int64 
 11  cplusplus            148 non-null    int64 
 12  css                  148 non-null    int64 
 13  dart                 148 non-null    int64 
 14  dataframe            148 non-null    int64 
 15  django               148 non-null    int64 
 16  docker  

In [8]:
old_col = [ x for x in df_topic_day['tag'].unique()]
new_col = [ x.replace('.', '').replace('-', '_').replace('+', 'plus') for x in df_topic_day['tag'].unique()]

In [9]:
col_dict = { x.replace('.', '').replace('-', '_').replace('+', 'plus') :x for x in df_topic_day['tag'].unique()}

In [17]:
# a.python_yn / a.tot              as python_yn,

' '.join(['CAST(a.'+x+'_yn AS NUMERIC)/ a.tot    as    '+x+'_yn , ' for x in sede_treat_0_1.columns[2:]])

'CAST(a.net_yn AS NUMERIC)/ a.tot    as    net_yn ,  CAST(a.amazon_web_services_yn AS NUMERIC)/ a.tot    as    amazon_web_services_yn ,  CAST(a.android_yn AS NUMERIC)/ a.tot    as    android_yn ,  CAST(a.angular_yn AS NUMERIC)/ a.tot    as    angular_yn ,  CAST(a.api_yn AS NUMERIC)/ a.tot    as    api_yn ,  CAST(a.arrays_yn AS NUMERIC)/ a.tot    as    arrays_yn ,  CAST(a.azure_yn AS NUMERIC)/ a.tot    as    azure_yn ,  CAST(a.c_yn AS NUMERIC)/ a.tot    as    c_yn ,  CAST(a.c#_yn AS NUMERIC)/ a.tot    as    c#_yn ,  CAST(a.cplusplus_yn AS NUMERIC)/ a.tot    as    cplusplus_yn ,  CAST(a.css_yn AS NUMERIC)/ a.tot    as    css_yn ,  CAST(a.dart_yn AS NUMERIC)/ a.tot    as    dart_yn ,  CAST(a.dataframe_yn AS NUMERIC)/ a.tot    as    dataframe_yn ,  CAST(a.django_yn AS NUMERIC)/ a.tot    as    django_yn ,  CAST(a.docker_yn AS NUMERIC)/ a.tot    as    docker_yn ,  CAST(a.excel_yn AS NUMERIC)/ a.tot    as    excel_yn ,  CAST(a.firebase_yn AS NUMERIC)/ a.tot    as    firebase_yn ,  CAST(a.flut

In [10]:
sede_treat = pd.concat([sede_treat_0_1, sede_treat_0_2, sede_treat_1_1, sede_treat_1_2], axis = 0)

In [11]:
sede_treat

,cdate,del_yn,net,amazon_web_services,android,angular,api,arrays,azure,c,...,reactjs,spring,spring_boot,sql,sql_server,swift,typescript,vba,visual_studio_code,vuejs
0,2021-10-08 00:00:00,N,89,58,140,81,23,63,46,58,...,213,31,53,95,44,55,96,30,25,62
1,2021-10-11 00:00:00,N,86,64,128,79,31,80,57,74,...,239,35,45,110,48,49,98,35,23,50
2,2021-10-14 00:00:00,N,91,58,133,94,30,87,64,79,...,228,46,55,130,45,55,108,33,26,67
3,2021-10-17 00:00:00,N,38,38,89,31,17,66,18,58,...,143,20,27,59,18,45,75,11,20,26
4,2021-11-09 00:00:00,N,76,48,161,108,19,114,49,71,...,240,37,64,153,57,56,122,40,44,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2023-01-22 00:00:00,Y,16,3,44,9,13,18,7,22,...,61,7,9,29,10,15,18,11,9,9
142,2023-01-25 00:00:00,Y,38,26,45,27,16,30,5,26,...,90,14,23,63,38,30,32,12,15,7
143,2023-02-17 00:00:00,Y,26,19,39,17,17,24,15,23,...,76,16,19,42,24,27,20,6,17,10
144,2023-02-20 00:00:00,Y,28,25,54,16,17,38,16,23,...,75,10,24,44,26,25,21,11,13,10


In [12]:
sede_treat.columns[2:]

Index(['net', 'amazon_web_services', 'android', 'angular', 'api', 'arrays',
       'azure', 'c', 'c#', 'cplusplus', 'css', 'dart', 'dataframe', 'django',
       'docker', 'excel', 'firebase', 'flutter', 'html', 'ios', 'java',
       'javascript', 'jquery', 'json', 'kotlin', 'laravel', 'linux', 'list',
       'mongodb', 'mysql', 'nextjs', 'nodejs', 'numpy', 'pandas', 'php',
       'postgresql', 'python', 'python_3x', 'r', 'react_native', 'reactjs',
       'spring', 'spring_boot', 'sql', 'sql_server', 'swift', 'typescript',
       'vba', 'visual_studio_code', 'vuejs'],
      dtype='object')

In [13]:
col_dict

{'net': '.net',
 'amazon_web_services': 'amazon-web-services',
 'android': 'android',
 'angular': 'angular',
 'api': 'api',
 'arrays': 'arrays',
 'azure': 'azure',
 'c': 'c',
 'c#': 'c#',
 'cplusplus': 'c++',
 'css': 'css',
 'dart': 'dart',
 'dataframe': 'dataframe',
 'django': 'django',
 'docker': 'docker',
 'excel': 'excel',
 'firebase': 'firebase',
 'flutter': 'flutter',
 'html': 'html',
 'ios': 'ios',
 'java': 'java',
 'javascript': 'javascript',
 'jquery': 'jquery',
 'json': 'json',
 'kotlin': 'kotlin',
 'laravel': 'laravel',
 'linux': 'linux',
 'list': 'list',
 'mongodb': 'mongodb',
 'mysql': 'mysql',
 'nextjs': 'next.js',
 'nodejs': 'node.js',
 'numpy': 'numpy',
 'pandas': 'pandas',
 'php': 'php',
 'postgresql': 'postgresql',
 'python': 'python',
 'python_3x': 'python-3.x',
 'r': 'r',
 'react_native': 'react-native',
 'reactjs': 'reactjs',
 'spring': 'spring',
 'spring_boot': 'spring-boot',
 'sql': 'sql',
 'sql_server': 'sql-server',
 'swift': 'swift',
 'typescript': 'typescript

In [14]:
for col_nm in sede_treat.columns[2:] : 
    print(col_nm)
    print(col_dict[col_nm])
    sede_treat.rename(columns = {col_nm: col_dict[col_nm]}, inplace=True)


net
.net
amazon_web_services
amazon-web-services
android
android
angular
angular
api
api
arrays
arrays
azure
azure
c
c
c#
c#
cplusplus
c++
css
css
dart
dart
dataframe
dataframe
django
django
docker
docker
excel
excel
firebase
firebase
flutter
flutter
html
html
ios
ios
java
java
javascript
javascript
jquery
jquery
json
json
kotlin
kotlin
laravel
laravel
linux
linux
list
list
mongodb
mongodb
mysql
mysql
nextjs
next.js
nodejs
node.js
numpy
numpy
pandas
pandas
php
php
postgresql
postgresql
python
python
python_3x
python-3.x
r
r
react_native
react-native
reactjs
reactjs
spring
spring
spring_boot
spring-boot
sql
sql
sql_server
sql-server
swift
swift
typescript
typescript
vba
vba
visual_studio_code
visual-studio-code
vuejs
vue.js


In [15]:
sede_treat.columns

Index(['cdate', 'del_yn', '.net', 'amazon-web-services', 'android', 'angular',
       'api', 'arrays', 'azure', 'c', 'c#', 'c++', 'css', 'dart', 'dataframe',
       'django', 'docker', 'excel', 'firebase', 'flutter', 'html', 'ios',
       'java', 'javascript', 'jquery', 'json', 'kotlin', 'laravel', 'linux',
       'list', 'mongodb', 'mysql', 'next.js', 'node.js', 'numpy', 'pandas',
       'php', 'postgresql', 'python', 'python-3.x', 'r', 'react-native',
       'reactjs', 'spring', 'spring-boot', 'sql', 'sql-server', 'swift',
       'typescript', 'vba', 'visual-studio-code', 'vue.js'],
      dtype='object')

In [16]:
sede_treat = sede_treat.melt(id_vars = ['cdate', 'del_yn'], value_vars=sede_treat.columns[1:])

In [17]:
sede_treat

,cdate,del_yn,variable,value
0,2021-10-08 00:00:00,N,.net,89
1,2021-10-11 00:00:00,N,.net,86
2,2021-10-14 00:00:00,N,.net,91
3,2021-10-17 00:00:00,N,.net,38
4,2021-11-09 00:00:00,N,.net,76
...,...,...,...,...
32195,2023-01-22 00:00:00,Y,vue.js,9
32196,2023-01-25 00:00:00,Y,vue.js,7
32197,2023-02-17 00:00:00,Y,vue.js,10
32198,2023-02-20 00:00:00,Y,vue.js,10


In [18]:
sede_treat['cdate'] = pd.to_datetime(sede_treat['cdate'])

In [19]:
sede_treat.columns = ['cdate', 'del_yn', 'sede_tag', 'sede_q_count']

In [20]:
sede_treat.loc[sede_treat['sede_tag'] =='net', 'sede_tag'] ='.net'

In [21]:
sede_treat_y = sede_treat.loc[sede_treat['del_yn'] =='Y', ['cdate', 'sede_tag', 'sede_q_count']]
sede_treat_n = sede_treat.groupby(['cdate', 'sede_tag']).sum()[['sede_q_count']].reset_index()

sede_treat_yn = sede_treat.groupby(['cdate', 'sede_tag']).sum()[['sede_q_count']].reset_index()

In [22]:
tt_del_n = pd.merge(df_topic_day, sede_treat, left_on = ['date', 'tag'], right_on = ['cdate', 'sede_tag'], how= 'left')
tt_del_y = pd.merge(df_topic_day, sede_treat, left_on = ['date', 'tag'], right_on = ['cdate', 'sede_tag'], how= 'left')

In [23]:
df_topic_day['date'] = pd.to_datetime(df_topic_day['date'])
tt['date'] = pd.to_datetime(tt['date'])

NameError: name 'tt' is not defined

In [ ]:
# rel_week을 범주형 변수로 변환 (주차별 처리)
df_topic_day['rel_week'] = pd.Categorical(df_topic_day['rel_week'])
tt['rel_week'] = pd.Categorical(tt['rel_week'])

In [ ]:
tt.head()

In [ ]:
#postswithdeleted에서는 .net 태그를 지우므로 분석에서 제외
df_topic_day = df_topic_day[df_topic_day['tag'] != '.net']
tt = tt[tt['tag'] != '.net']

In [ ]:
df_topic_day = df_topic_day.set_index(['tag', 'date'])
tt = tt.set_index(['tag', 'date'])

In [ ]:
# post와 treat 상호작용 변수 추가 (R의 i(post, ref=0):treat 대응)
# df_topic_day['post_treat'] = df_topic_day['post'] * df_topic_day['treat']

In [ ]:
# 고정 효과 모델 (tag를 고정 효과로 포함하고, rel_week도 설명 변수로 포함)
mod_quest_vol_so_dyn = PanelOLS.from_formula('q_count ~ C(rel_week) * treat + EntityEffects',
                                             data=df_topic_day)
result_dyn = mod_quest_vol_so_dyn.fit(cov_type='clustered', cluster_entity=True)


In [ ]:
# 고정 효과 모델 (tag를 고정 효과로 포함하고, rel_week도 설명 변수로 포함)
tt_mod_quest_vol_so_dyn = PanelOLS.from_formula('sede_q_count ~ C(rel_week) * treat + EntityEffects',
                                             data=tt)
tt_result_dyn = tt_mod_quest_vol_so_dyn.fit(cov_type='clustered', cluster_entity=True)


In [ ]:
# 동적 효과 계수 추출
coefs = result_dyn.params
std_errors = result_dyn.std_errors
conf_int = result_dyn.conf_int()

In [ ]:
# 동적 효과 계수 추출
tt_coefs = tt_result_dyn.params
tt_std_errors = tt_result_dyn.std_errors
tt_conf_int = tt_result_dyn.conf_int()

In [ ]:
periods = np.arange(-7, 15, 1)


In [ ]:
coefs['C(rel_week)[T.14]:treat']
#     'C(rel_week)[T.14]:treat'

In [ ]:
coefs_filtered = coefs.loc[[f'C(rel_week)[T.{i}]:treat' for i in periods]].values
std_errors_filtered = std_errors.loc[[f'C(rel_week)[T.{i}]:treat' for i in periods]].values


In [ ]:
tt_coefs_filtered = tt_coefs.loc[[f'C(rel_week)[T.{i}]:treat' for i in periods]].values
tt_std_errors_filtered = tt_std_errors.loc[[f'C(rel_week)[T.{i}]:treat' for i in periods]].values


In [ ]:
# 신뢰구간 계산
ll_95 = coefs_filtered - 1.965 * std_errors_filtered
ul_95 = coefs_filtered + 1.965 * std_errors_filtered

# 데이터프레임 생성 (계수와 신뢰구간)
df_coefs = pd.DataFrame({
    'period': periods,
    'Estimate': coefs_filtered,
    'll_95': ll_95,
    'ul_95': ul_95
})

In [ ]:
# 신뢰구간 계산
tt_ll_95 = tt_coefs_filtered - 1.965 * tt_std_errors_filtered
tt_ul_95 = tt_coefs_filtered + 1.965 * tt_std_errors_filtered

# 데이터프레임 생성 (계수와 신뢰구간)
tt_df_coefs = pd.DataFrame({
    'period': periods,
    'Estimate': tt_coefs_filtered,
    'll_95': tt_ll_95,
    'ul_95': tt_ul_95
})

In [ ]:
### create a figure with multiple subplots
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(1, 2, figsize = (15, 5), sharey=sharey, sharex=sharex)


axs[0].fill_between(df_coefs['period'], df_coefs['ll_95'], df_coefs['ul_95'], color='gray', alpha=0.5)
axs[0].plot(df_coefs['period'], df_coefs['Estimate'], color='black', linewidth=2)
axs[0].axvline(x=0, color='red', linestyle='-.', linewidth=1)
axs[0].axhline(y=0, color='black', linestyle='--')
axs[0].set_xlabel("Week Relative to ChatGPT Release", fontsize = 12.5)
axs[0].set_ylabel("Content Volume Effect and 95% CI", fontsize = 12.5)
axs[0].set_title("Stack Overflow(survived only)", fontsize=16)
axs[0].tick_params(axis='x', rotation=45)
axs[0].grid(False)


axs[1].fill_between(tt_df_coefs['period'], tt_df_coefs['ll_95'], tt_df_coefs['ul_95'], color='gray', alpha=0.5)
axs[1].plot(tt_df_coefs['period'], tt_df_coefs['Estimate'], color='black', linewidth=2)
axs[1].axvline(x=0, color='red', linestyle='-.', linewidth=1)
axs[1].axhline(y=0, color='black', linestyle='--')
axs[1].axvline(x=3, color='blue', linestyle='-.', linewidth=1)

axs[1].set_xlabel("Week Relative to ChatGPT Release", fontsize = 12.5)
axs[1].set_ylabel("Content Volume Effect and 95% CI", fontsize = 12.5)
axs[1].set_title("Stack Overflow(survived + deleted)", fontsize=16)
axs[1].tick_params(axis='x', rotation=45)
axs[1].grid(False)

plt.show();

In [ ]:
tt_df_coefs

In [ ]:
df_coefs

tt.to_csv('chk.csv')

In [ ]:
tt.to_csv('chk.csv')

In [ ]:
tt = tt.reset_index()

In [ ]:
ttt = tt.groupby(['tag', 'treat']).sum([['q_count', 'sede_q_count']]).reset_index()

In [ ]:
ttt = ttt[['tag', 'treat', 'q_count', 'sede_q_count']]

In [ ]:
ttt = ttt.pivot_table(index = ['tag'], columns=['treat'], values = ['q_count', 'sede_q_count']).reset_index()

In [ ]:
ttt['q_rate'] = (ttt[('q_count', 1)] - ttt[('q_count', 0)])/ttt[('q_count', 0)]*100
ttt['sede_q_rate'] = (ttt[('sede_q_count', 1)] - ttt[('sede_q_count', 0)])/ttt[('sede_q_count', 0)]*100

In [ ]:
ttt['diff'] = ttt['sede_q_rate'] - ttt['q_rate']

In [ ]:
ttt['diff']

In [ ]:
ttt